In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np
import pandas as pd
import torch.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder

In [7]:
data = pd.read_csv('C:\\Users\\hkimr\\Desktop\\REU Day 1\\Aggregated User Data.csv')
data.head()

,UUID,timestamp,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,...,label:STAIRS_-_GOING_DOWN,label:ELEVATOR,label:OR_standing,label:AT_SCHOOL,label:PHONE_IN_HAND,label:PHONE_IN_BAG,label:PHONE_ON_TABLE,label:WITH_CO-WORKERS,label:WITH_FRIENDS,label_source
0,00EABED2-271D-49D8-B599-1D4A09240601,1444079161,0.996815,0.003529,-0.002786,0.006496,0.995203,0.996825,0.998502,1.748756,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2
1,00EABED2-271D-49D8-B599-1D4A09240601,1444079221,0.996864,0.004172,-0.003110,0.007050,0.994957,0.996981,0.998766,1.935573,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2
2,00EABED2-271D-49D8-B599-1D4A09240601,1444079281,0.996825,0.003667,0.003094,0.006076,0.994797,0.996614,0.998704,2.031780,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2
3,00EABED2-271D-49D8-B599-1D4A09240601,1444079341,0.996874,0.003541,0.000626,0.006059,0.995050,0.996907,0.998690,1.865318,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2
4,00EABED2-271D-49D8-B599-1D4A09240601,1444079431,0.997371,0.037653,0.043389,0.102332,0.995548,0.996860,0.998205,0.460806,...,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1.0,NaN,2


In [8]:
data = data[(data['UUID'] == '0BFC35E2-4817-4865-BFA7-764742302A2D') | (data['UUID'] == '0A986513-7828-4D53-AA1F-E02D6DF9561B') | (data['UUID'] == '00EABED2-271D-49D8-B599-1D4A09240601')] 
data.drop(columns = ['timestamp'], inplace = True)

# Creating a new dataframe with only sitting, walking, and sleeping data

In [9]:
only_walking = data[(data['label:FIX_walking'] == 1) & (data['label:SITTING'] == 0) & (data['label:SLEEPING'] == 0)]
only_walking = only_walking.iloc[:,1:27]
only_walking['label'] = "WALKING"

only_sitting = data[(data['label:FIX_walking'] == 0) & (data['label:SITTING'] == 1) & (data['label:SLEEPING'] == 0)]
only_sitting = only_sitting.iloc[:,1:27]
only_sitting['label'] = "SITTING"

only_sleeping = data[(data['label:FIX_walking'] == 0) & (data['label:SITTING'] == 0) & (data['label:SLEEPING'] == 1)]
only_sleeping = only_sleeping.iloc[:,1:27]
only_sleeping['label'] = "SLEEPING"

df = pd.concat([only_walking, only_sitting, only_sleeping], axis = 0)
df.head()

,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,raw_acc:magnitude_spectrum:log_energy_band0,...,raw_acc:3d:mean_x,raw_acc:3d:mean_y,raw_acc:3d:mean_z,raw_acc:3d:std_x,raw_acc:3d:std_y,raw_acc:3d:std_z,raw_acc:3d:ro_xy,raw_acc:3d:ro_xz,raw_acc:3d:ro_yz,label
155,1.003052,0.139768,0.115192,0.273656,0.991920,0.996238,0.999979,1.328158,6.674634,5.027699,...,-0.051719,-0.168622,-0.866269,0.250667,0.343037,0.252212,-0.030830,0.143926,-0.530033,WALKING
156,1.005091,0.232927,0.190056,0.292741,0.826441,0.948630,1.176819,2.745002,6.658351,5.013784,...,0.255740,-0.433965,-0.801786,0.276395,0.190099,0.235251,0.105712,0.016350,0.027738,WALKING
157,0.988339,0.025706,-0.012189,0.044177,0.979990,0.987836,0.996713,1.783550,6.684273,5.043187,...,-0.028900,-0.594037,-0.772274,0.136729,0.065971,0.065611,-0.603787,0.630796,-0.799983,WALKING
158,1.005461,0.105208,0.128558,0.199233,0.992822,0.994770,0.997684,1.459081,6.679381,5.039396,...,0.152049,-0.164488,-0.791628,0.272913,0.297443,0.426869,-0.372997,0.652161,-0.524372,WALKING
159,1.001464,0.121184,0.129053,0.209801,0.942805,0.996160,1.052143,2.060366,6.677517,5.029112,...,0.149694,-0.014688,-0.913222,0.299310,0.237561,0.122455,0.601753,-0.072091,-0.003273,WALKING


# Interpolating acceleration columns with average values

In [10]:
def interpolation(df):
    col_to_avg = list(df.columns) #Start with keeping all the columns as columns to use an average interpolation on
    for k in range(len(list(df.columns))):
        if list(df.columns)[k].startswith(('discrete', 'label')): #Remove label and discrete columns from col_to_avg
            col_to_avg.remove(list(df.columns)[k])
    
    df_with_avg = df[col_to_avg].fillna(df[col_to_avg].mean()) #Interpolate nan columns for all continuous-valued columns with average
    
    col_to_zero = list(df.columns)
    for k in range(len(list(df.columns))):
        if not list(df.columns)[k].startswith(('discrete', 'label')): #Remove all columns except label and discrete
            col_to_zero.remove(list(df.columns)[k])
    
    df_with_zero = df[col_to_zero].fillna(0) #Interpolate nan values for label and discrete columns with 0
    
    return pd.concat([df_with_avg, df_with_zero], axis = 1)

In [11]:
df.iloc[:,:-1] = interpolation(df.iloc[:,:-1]) #Interpolate acceleration columns
df = df.values
df

array([[1.003052, 0.139768, 0.115192, ..., 0.143926, -0.530033,
        'WALKING'],
       [1.005091, 0.232927, 0.190056, ..., 0.01635, 0.027738, 'WALKING'],
       [0.988339, 0.025706, -0.012189, ..., 0.630796, -0.799983,
        'WALKING'],
       ...,
       [1.000489, 0.001597, 0.000608, ..., -0.131635, 0.020957,
        'SLEEPING'],
       [1.000168, 0.003089, -0.0016879999999999998, ...,
        -0.08845900000000001, -0.090014, 'SLEEPING'],
       [1.019187, 0.143322, 0.24915, ..., 0.19007, -0.548567, 'SLEEPING']],
      dtype=object)

In [12]:
X = df[:, :-1]
y = df[:, -1]

y = LabelEncoder().fit_transform(y)

# SMOTE Oversampling

In [13]:
largest_volume = len(only_sitting)

In [14]:
oversample = SMOTE(sampling_strategy = {0:largest_volume, 1: int(largest_volume * 0.9), 2: int(largest_volume * 0.9)})
X, y = oversample.fit_resample(X, y)

In [15]:
X.shape ### all classes are now 90% the size of the largest class

(9405, 26)

# Train/Test Split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train

array([[-0.65473869, -0.38170131, -0.55182899, ..., -0.70895551,
         0.07953468, -1.72655411],
       [-0.31283429, -0.53203596, -0.50299507, ...,  0.06718223,
        -0.76044329, -0.27941755],
       [-0.49887932, -0.53262336, -0.49164275, ...,  1.25279561,
        -1.32074595, -1.06918716],
       ...,
       [-0.36864147, -0.51142164, -0.43577665, ..., -0.86387876,
        -2.32493672,  1.38866375],
       [-0.2500982 , -0.53077587, -0.47637227, ..., -0.02524002,
         0.7799074 ,  0.50122684],
       [-0.41958426, -0.53826353, -0.50131137, ...,  0.02763833,
         0.29844514,  0.1338062 ]])

# Defining the model

In [17]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Classifier(nn.Module):
    def __init__(self, feature_dim = 26):
        super(Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 20),
            classifier_block(20, 15),
            classifier_block(15, 10),
            classifier_block(10, 5),
            nn.Linear(5, 3)
        )
    def forward(self, x):
        #softmax = nn.Softmax(dim = 1)
        return self.network(x)

# Hyperparameters

In [18]:
model = Classifier()
lr = 0.01
n_epochs = 5000
batch_size = 100

### 2000 epochs 75 batch size 0.001 learning rate
#criterion = FocalLoss(alpha = 0.2, gamma = 5)

### Class 0 --> Sitting, Class 1 --> Sleeping, Class 2 --> Walking 
# weights = torch.tensor([1.0, 2.07, 13], dtype=torch.float32)
# weights = weights / weights.sum()
# normedWeights = torch.FloatTensor(weights)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = lr)

train_features = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
test_features = torch.tensor(X_test)
test_labels = torch.tensor(y_test)

train_data = torch.utils.data.TensorDataset(train_features, train_labels)
test_data = torch.utils.data.TensorDataset(test_features, test_labels)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = len(test_labels), shuffle = True)

# Training Loop

In [19]:
for epoch in range(n_epochs):
    total_loss = 0
    for batch in train_loader:
        features, labels = batch
        #_, encoded_labels = torch.max(labels, dim = 1) 
        
        optimizer.zero_grad()
        preds = model(features.float())
        
        loss = criterion(preds, labels.long()) 
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 1.0)
        optimizer.step()
        total_loss += loss.item()
        
    print(f'Epoch {epoch + 1}, Loss: {total_loss}, Final Batch Loss: {loss.item()}')
#     softmax = nn.Softmax(dim = 1)
#     _, preds = torch.max(softmax(model(features.float())), dim = 1)
#     _, answers = torch.max(labels, dim = 1)
#     print(metrics.confusion_matrix(answers, preds))

Epoch 1, Loss: 61.14700800180435, Final Batch Loss: 0.563912570476532
Epoch 2, Loss: 43.466257989406586, Final Batch Loss: 0.4089049994945526
Epoch 3, Loss: 37.67613685131073, Final Batch Loss: 0.3382885754108429
Epoch 4, Loss: 35.754526883363724, Final Batch Loss: 0.3622346818447113
Epoch 5, Loss: 34.29636999964714, Final Batch Loss: 0.3721713721752167
Epoch 6, Loss: 34.539160430431366, Final Batch Loss: 0.32628169655799866
Epoch 7, Loss: 33.78323656320572, Final Batch Loss: 0.3790557384490967
Epoch 8, Loss: 33.05526986718178, Final Batch Loss: 0.38833168148994446
Epoch 9, Loss: 32.37554806470871, Final Batch Loss: 0.4057650566101074
Epoch 10, Loss: 31.150546967983246, Final Batch Loss: 0.4033285677433014
Epoch 11, Loss: 31.498560577630997, Final Batch Loss: 0.5220275521278381
Epoch 12, Loss: 30.191804379224777, Final Batch Loss: 0.17878974974155426
Epoch 13, Loss: 31.45393458008766, Final Batch Loss: 0.3312879204750061
Epoch 14, Loss: 31.42789340019226, Final Batch Loss: 0.3374840021

Epoch 114, Loss: 23.99711911380291, Final Batch Loss: 0.15733692049980164
Epoch 115, Loss: 23.901661083102226, Final Batch Loss: 0.6060308218002319
Epoch 116, Loss: 24.018264800310135, Final Batch Loss: 0.33429086208343506
Epoch 117, Loss: 24.170999616384506, Final Batch Loss: 0.25685814023017883
Epoch 118, Loss: 23.715244248509407, Final Batch Loss: 0.3394480049610138
Epoch 119, Loss: 23.96926249563694, Final Batch Loss: 0.3223251700401306
Epoch 120, Loss: 24.708617970347404, Final Batch Loss: 0.3998318016529083
Epoch 121, Loss: 23.596461713314056, Final Batch Loss: 0.21100221574306488
Epoch 122, Loss: 24.077099531888962, Final Batch Loss: 0.2826695740222931
Epoch 123, Loss: 23.909970343112946, Final Batch Loss: 0.18140137195587158
Epoch 124, Loss: 24.01675769686699, Final Batch Loss: 0.18707947432994843
Epoch 125, Loss: 23.512329056859016, Final Batch Loss: 0.3176083266735077
Epoch 126, Loss: 23.87589982151985, Final Batch Loss: 0.5164629220962524
Epoch 127, Loss: 23.511522755026817,

Epoch 225, Loss: 22.47284258902073, Final Batch Loss: 0.4910620450973511
Epoch 226, Loss: 21.58368369936943, Final Batch Loss: 0.3649853765964508
Epoch 227, Loss: 22.73156525194645, Final Batch Loss: 0.22204434871673584
Epoch 228, Loss: 22.518824979662895, Final Batch Loss: 0.5169049501419067
Epoch 229, Loss: 23.136989071965218, Final Batch Loss: 0.29902443289756775
Epoch 230, Loss: 22.60507522523403, Final Batch Loss: 0.29965975880622864
Epoch 231, Loss: 22.77655564248562, Final Batch Loss: 0.35251152515411377
Epoch 232, Loss: 22.77411776781082, Final Batch Loss: 0.4279678165912628
Epoch 233, Loss: 22.453123047947884, Final Batch Loss: 0.41761890053749084
Epoch 234, Loss: 22.145058557391167, Final Batch Loss: 0.20157845318317413
Epoch 235, Loss: 22.87221910059452, Final Batch Loss: 0.48302045464515686
Epoch 236, Loss: 22.921272486448288, Final Batch Loss: 0.36880943179130554
Epoch 237, Loss: 22.155882343649864, Final Batch Loss: 0.3012230098247528
Epoch 238, Loss: 22.62788313627243, F

Epoch 336, Loss: 21.299093782901764, Final Batch Loss: 0.20394140481948853
Epoch 337, Loss: 22.018194243311882, Final Batch Loss: 0.23657454550266266
Epoch 338, Loss: 22.039283871650696, Final Batch Loss: 0.2756311595439911
Epoch 339, Loss: 22.49829901754856, Final Batch Loss: 0.4434969425201416
Epoch 340, Loss: 21.40196219086647, Final Batch Loss: 0.14597459137439728
Epoch 341, Loss: 22.390053778886795, Final Batch Loss: 0.38313665986061096
Epoch 342, Loss: 21.71073331683874, Final Batch Loss: 0.11218766123056412
Epoch 343, Loss: 21.588209450244904, Final Batch Loss: 0.16458825767040253
Epoch 344, Loss: 21.58568276464939, Final Batch Loss: 0.47584614157676697
Epoch 345, Loss: 22.147451549768448, Final Batch Loss: 0.20681458711624146
Epoch 346, Loss: 22.231113448739052, Final Batch Loss: 0.42402175068855286
Epoch 347, Loss: 22.31262555718422, Final Batch Loss: 0.5151830315589905
Epoch 348, Loss: 21.625778153538704, Final Batch Loss: 0.20336540043354034
Epoch 349, Loss: 22.4078020304441

Epoch 447, Loss: 21.613763198256493, Final Batch Loss: 0.31823793053627014
Epoch 448, Loss: 20.891756929457188, Final Batch Loss: 0.3013743758201599
Epoch 449, Loss: 21.145382314920425, Final Batch Loss: 0.36948490142822266
Epoch 450, Loss: 21.189471408724785, Final Batch Loss: 0.1239219605922699
Epoch 451, Loss: 20.957524836063385, Final Batch Loss: 0.20988678932189941
Epoch 452, Loss: 21.119350597262383, Final Batch Loss: 0.228271484375
Epoch 453, Loss: 21.188223838806152, Final Batch Loss: 0.2899807095527649
Epoch 454, Loss: 20.461200013756752, Final Batch Loss: 0.31123650074005127
Epoch 455, Loss: 20.421710461378098, Final Batch Loss: 0.24975287914276123
Epoch 456, Loss: 21.218298599123955, Final Batch Loss: 0.29896268248558044
Epoch 457, Loss: 21.386135160923004, Final Batch Loss: 0.1712009459733963
Epoch 458, Loss: 21.495386838912964, Final Batch Loss: 0.5431851744651794
Epoch 459, Loss: 21.798112839460373, Final Batch Loss: 0.5790738463401794
Epoch 460, Loss: 20.349653884768486,

...

Epoch 4490, Loss: 16.320466846227646, Final Batch Loss: 0.2179718017578125
Epoch 4491, Loss: 16.055594347417355, Final Batch Loss: 0.05802629888057709
Epoch 4492, Loss: 15.613068282604218, Final Batch Loss: 0.28250277042388916
Epoch 4493, Loss: 15.922228731215, Final Batch Loss: 0.2605556845664978
Epoch 4494, Loss: 15.639303907752037, Final Batch Loss: 0.11255663633346558
Epoch 4495, Loss: 15.774696461856365, Final Batch Loss: 0.2930265963077545
Epoch 4496, Loss: 17.329942755401134, Final Batch Loss: 0.24777163565158844
Epoch 4497, Loss: 16.60885151475668, Final Batch Loss: 0.3561215400695801
Epoch 4498, Loss: 15.871032603085041, Final Batch Loss: 0.20455308258533478
Epoch 4499, Loss: 17.088376946747303, Final Batch Loss: 0.0514649897813797
Epoch 4500, Loss: 15.970793955028057, Final Batch Loss: 0.42290031909942627
Epoch 4501, Loss: 15.90404561907053, Final Batch Loss: 0.1392350047826767
Epoch 4502, Loss: 16.520146630704403, Final Batch Loss: 0.13827574253082275
Epoch 4503, Loss: 15.79

Epoch 4599, Loss: 16.46870892494917, Final Batch Loss: 0.17127537727355957
Epoch 4600, Loss: 15.124015856534243, Final Batch Loss: 0.20587463676929474
Epoch 4601, Loss: 15.671792432665825, Final Batch Loss: 0.09440024942159653
Epoch 4602, Loss: 16.17932765930891, Final Batch Loss: 0.20322193205356598
Epoch 4603, Loss: 16.55871158838272, Final Batch Loss: 0.26107874512672424
Epoch 4604, Loss: 16.27575583010912, Final Batch Loss: 0.08492930978536606
Epoch 4605, Loss: 18.24213634431362, Final Batch Loss: 0.1058460995554924
Epoch 4606, Loss: 15.730790629982948, Final Batch Loss: 0.10323059558868408
Epoch 4607, Loss: 15.999655686318874, Final Batch Loss: 0.04571312665939331
Epoch 4608, Loss: 15.92892549186945, Final Batch Loss: 0.15383799374103546
Epoch 4609, Loss: 15.314730290323496, Final Batch Loss: 0.06523675471544266
Epoch 4610, Loss: 16.421898372471333, Final Batch Loss: 0.2152298539876938
Epoch 4611, Loss: 16.88173808902502, Final Batch Loss: 0.2440740317106247
Epoch 4612, Loss: 16.8

Epoch 4707, Loss: 15.60077365487814, Final Batch Loss: 0.20489300787448883
Epoch 4708, Loss: 16.521428622305393, Final Batch Loss: 0.13506150245666504
Epoch 4709, Loss: 16.273934483528137, Final Batch Loss: 0.12244560569524765
Epoch 4710, Loss: 15.019337370991707, Final Batch Loss: 0.07096251100301743
Epoch 4711, Loss: 15.871007733047009, Final Batch Loss: 0.25260236859321594
Epoch 4712, Loss: 17.048091016709805, Final Batch Loss: 0.5218997597694397
Epoch 4713, Loss: 15.57608150690794, Final Batch Loss: 0.2665306627750397
Epoch 4714, Loss: 14.772175289690495, Final Batch Loss: 0.321754515171051
Epoch 4715, Loss: 16.536747187376022, Final Batch Loss: 0.2835724651813507
Epoch 4716, Loss: 15.933348499238491, Final Batch Loss: 0.12673301994800568
Epoch 4717, Loss: 14.640283592045307, Final Batch Loss: 0.07875661551952362
Epoch 4718, Loss: 15.795722708106041, Final Batch Loss: 0.23358875513076782
Epoch 4719, Loss: 16.05384761095047, Final Batch Loss: 0.12079357355833054
Epoch 4720, Loss: 16

Epoch 4816, Loss: 15.800627246499062, Final Batch Loss: 0.5890615582466125
Epoch 4817, Loss: 15.705758601427078, Final Batch Loss: 0.08886600285768509
Epoch 4818, Loss: 15.855795785784721, Final Batch Loss: 0.10779344290494919
Epoch 4819, Loss: 16.566578842699528, Final Batch Loss: 0.1732911318540573
Epoch 4820, Loss: 15.735775992274284, Final Batch Loss: 0.1451743245124817
Epoch 4821, Loss: 15.96311303973198, Final Batch Loss: 0.2963479459285736
Epoch 4822, Loss: 15.328277722001076, Final Batch Loss: 0.27572667598724365
Epoch 4823, Loss: 15.815755642950535, Final Batch Loss: 0.1740928292274475
Epoch 4824, Loss: 14.808047737926245, Final Batch Loss: 0.06208989396691322
Epoch 4825, Loss: 15.396133810281754, Final Batch Loss: 0.1659134328365326
Epoch 4826, Loss: 15.377256840467453, Final Batch Loss: 0.21702171862125397
Epoch 4827, Loss: 16.80494773387909, Final Batch Loss: 0.1495824009180069
Epoch 4828, Loss: 16.206114307045937, Final Batch Loss: 0.2656875550746918
Epoch 4829, Loss: 15.8

Epoch 4925, Loss: 15.523177668452263, Final Batch Loss: 0.1270708590745926
Epoch 4926, Loss: 15.649014689028263, Final Batch Loss: 0.13020464777946472
Epoch 4927, Loss: 16.241476953029633, Final Batch Loss: 0.2542566657066345
Epoch 4928, Loss: 15.923810459673405, Final Batch Loss: 0.1874232441186905
Epoch 4929, Loss: 15.43778233975172, Final Batch Loss: 0.5989459156990051
Epoch 4930, Loss: 16.215740270912647, Final Batch Loss: 0.12725897133350372
Epoch 4931, Loss: 15.510850861668587, Final Batch Loss: 0.11327660828828812
Epoch 4932, Loss: 15.418823599815369, Final Batch Loss: 0.1466522067785263
Epoch 4933, Loss: 16.12047665566206, Final Batch Loss: 0.18153254687786102
Epoch 4934, Loss: 15.816334269940853, Final Batch Loss: 0.31564876437187195
Epoch 4935, Loss: 15.507309913635254, Final Batch Loss: 0.39966437220573425
Epoch 4936, Loss: 15.966955602169037, Final Batch Loss: 0.19583575427532196
Epoch 4937, Loss: 15.617877125740051, Final Batch Loss: 0.08413781970739365
Epoch 4938, Loss: 1

# Evaluation on test data

In [20]:
softmax = nn.Softmax(dim = 1)
for batch in test_loader:
    features, labels = batch
    _, preds = torch.max(softmax(model(features.float())), dim = 1)
    #_, answers = torch.max(labels, dim = 1)
    print(metrics.confusion_matrix(labels, preds))
    print(metrics.classification_report(labels, preds, digits = 3))

[[555  38  66]
 [ 24 579   6]
 [ 28   3 582]]
              precision    recall  f1-score   support

           0      0.914     0.842     0.877       659
           1      0.934     0.951     0.942       609
           2      0.890     0.949     0.919       613

    accuracy                          0.912      1881
   macro avg      0.913     0.914     0.913      1881
weighted avg      0.913     0.912     0.912      1881



In [21]:
torch.save(model.state_dict(), f"../saved_models/5000Epochs_0.01LR_MutualExclusiveLabelClassifier")